In this notebook we explore different ways of adding noise to the circuits.

In [40]:
import numpy as np
import sympy
import cirq
import tensorflow_quantum as tfq
from tqdm import tqdm
import tensorflow as tf

from utilities.variational import VQE
from utilities.circuit_basics import Evaluator
from utilities.idinserter import IdInserter
from utilities.simplifier import Simplifier
from utilities.unitary_killer import UnitaryMurder

n_qubits = 4
qlr = 0.01
qepochs = 10**3
verbose=0
g=1
J=0
noise=0.0
problem="TFIM"
vqe_handler = VQE(n_qubits=n_qubits, lr=qlr, epochs=qepochs, patience=100,
                  random_perturbations=True, verbose=verbose, g=g, J = J, noise=noise, problem=problem)

iid = IdInserter(n_qubits=n_qubits)
Simp = Simplifier(n_qubits=n_qubits)
killer = UnitaryMurder(vqe_handler)
indexed_circuit=[vqe_handler.number_of_cnots+k for k in range(vqe_handler.n_qubits,2*vqe_handler.n_qubits)]

In [41]:
circuit, symbols, idx_symbols = vqe_handler.give_circuit(indexed_circuit) 
circuit

(0, 0): ───Rx(th_0)───

(0, 1): ───Rx(th_1)───

(0, 2): ───Rx(th_2)───

(0, 3): ───Rx(th_3)───

In [42]:
for q in vqe_handler.qubits:
    circuit.append(cirq.DepolarizingChannel(0.2).on(q))

In [43]:
circuit

(0, 0): ───Rx(th_0)───D(0.2)───

(0, 1): ───Rx(th_1)───D(0.2)───

(0, 2): ───Rx(th_2)───D(0.2)───

(0, 3): ───Rx(th_3)───D(0.2)───

In [26]:
circuit
dens = cirq.DensityMatrixSimulator(noise=cirq.depolarize(0.00))
result = dens.simulate(cirq.resolve_parameters(circuit, {s:k for s,k in zip(symbols, range(len(symbols)))}))
rho_dm = result.final_density_matrix

In [44]:
circuit.with_noise(cirq.DepolarizingChannel(0.01))

(0, 0): ───Rx(th_0)───D(0.01)[<virtual>]───D(0.2)───D(0.01)[<virtual>]───

(0, 1): ───Rx(th_1)───D(0.01)[<virtual>]───D(0.2)───D(0.01)[<virtual>]───

(0, 2): ───Rx(th_2)───D(0.01)[<virtual>]───D(0.2)───D(0.01)[<virtual>]───

(0, 3): ───Rx(th_3)───D(0.01)[<virtual>]───D(0.2)───D(0.01)[<virtual>]───